In [1]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import timeit
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
import tensorflow_model_optimization as tfmot
from tensorflow.keras.models import load_model
import os


device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

2023-05-29 18:25:54.293983: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-29 18:25:54.448870: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-29 18:25:54.836531: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/themandalorian/anaconda3/envs/ml-on-mc/lib/
2023-05-29 18:25:54.836582: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: lib

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Load the CIFAR-10 dataset
data_path = 'cifar-10-batches-py/'
train_data = []
train_labels = []
for i in range(1, 6):
    batch = unpickle(data_path + 'data_batch_' + str(i))
    train_data.append(batch[b'data'])
    train_labels += batch[b'labels']
    
    
train_data = np.concatenate(train_data)
train_data = train_data.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
test_data = unpickle(data_path + 'test_batch')[b'data']
test_data = test_data.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
train_labels = np.array(train_labels)
test_labels = np.array(unpickle(data_path + 'test_batch')[b'labels'])

# Print the number of features, classes, and training samples for CIFAR-10
print("CIFAR-10")
print("Number of features:", train_data.shape[1:])
print("Number of classes:", len(np.unique(train_labels)))
print("Number of training samples:", train_data.shape[0])

# Shuffle your dataset
train_data, train_labels = shuffle(train_data, train_labels, random_state=42)
test_data, test_labels = shuffle(test_data, test_labels, random_state=42)

# Normalize pixel values
x_train = train_data.astype('float32') / 255.0
x_test = test_data.astype('float32') / 255.0

# Convert labels to categorical format
y_train = to_categorical(train_labels, num_classes=10)
y_test = to_categorical(test_labels, num_classes=10)

# create data generator
datagen = ImageDataGenerator(zoom_range=[0.9,1.1],
                             rotation_range=30,
                             brightness_range=[0.9,1.1],
                             width_shift_range=0.3, 
                             height_shift_range=0.3,
                             vertical_flip=True,
                             horizontal_flip=True)
# prepare iterator
it_train = datagen.flow(x_train, y_train, batch_size=64)
# fit model
steps = int(x_train.shape[0] / 64)

CIFAR-10
Number of features: (32, 32, 3)
Number of classes: 10
Number of training samples: 50000


In [3]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

    c_str = ''

    # Create header guard
    c_str += '#ifndef ' + var_name.upper() + '_H\n'
    c_str += '#define ' + var_name.upper() + '_H\n\n'

    # Add array length at top of file
    c_str += '\nstatic const unsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

    # Declare C variable
    c_str += 'static const unsigned char ' + var_name + '[] = {'
    hex_array = []
    for i, val in enumerate(hex_data) :

        # Construct string from hex
        hex_str = format(val, '#04x')

        # Add formatting so each line stays within 80 characters
        if (i + 1) < len(hex_data):
            hex_str += ','
        if (i + 1) % 12 == 0:
            hex_str += '\n '
        hex_array.append(hex_str)

    # Add closing brace
    c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

    # Close out header guard
    c_str += '#endif //' + var_name.upper() + '_H'

    return c_str

In [107]:
model_10 = Sequential()
model_10.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

model_10.summary()

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_tiny_Base-CubeAI_6Conv.h5', save_freq='epoch')

# compile model
opt = SGD(lr=0.001, momentum=0.9)
model_10.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_89"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_537 (Conv2D)         (None, 32, 32, 8)         224       
                                                                 
 conv2d_538 (Conv2D)         (None, 32, 32, 8)         584       
                                                                 
 max_pooling2d_275 (MaxPooli  (None, 16, 16, 8)        0         
 ng2D)                                                           
                                                                 
 dropout_359 (Dropout)       (None, 16, 16, 8)         0         
                                                                 
 conv2d_539 (Conv2D)         (None, 16, 16, 16)        1168      
                                                                 
 conv2d_540 (Conv2D)         (None, 16, 16, 16)        2320      
                                                     

In [108]:
# Model Fit
model_10.fit(x_train,y_train,batch_size=64, epochs=1000, validation_data=(x_test, y_test),callbacks=[checkpoint_callback])

Epoch 1/1000
782/782 [==============================] - 4s 5ms/step - loss: 2.3103 - accuracy: 0.1105 - val_loss: 2.2917 - val_accuracy: 0.1849
Epoch 2/1000
782/782 [==============================] - 3s 4ms/step - loss: 2.2444 - accuracy: 0.1538 - val_loss: 2.1618 - val_accuracy: 0.1987
Epoch 3/1000
782/782 [==============================] - 4s 5ms/step - loss: 2.1099 - accuracy: 0.1811 - val_loss: 2.0013 - val_accuracy: 0.2412
Epoch 4/1000
782/782 [==============================] - 4s 5ms/step - loss: 2.0226 - accuracy: 0.1994 - val_loss: 1.9743 - val_accuracy: 0.2414
Epoch 5/1000
782/782 [==============================] - 4s 5ms/step - loss: 1.9756 - accuracy: 0.2207 - val_loss: 1.9278 - val_accuracy: 0.2653
Epoch 6/1000
782/782 [==============================] - 4s 5ms/step - loss: 1.9258 - accuracy: 0.2462 - val_loss: 1.8752 - val_accuracy: 0.2785
Epoch 7/1000
782/782 [==============================] - 4s 5ms/step - loss: 1.8788 - accuracy: 0.2674 - val_loss: 1.8114 - val_accuracy:

782/782 [==============================] - 4s 5ms/step - loss: 1.1352 - accuracy: 0.5946 - val_loss: 0.9855 - val_accuracy: 0.6514
Epoch 114/1000
782/782 [==============================] - 4s 5ms/step - loss: 1.1299 - accuracy: 0.5963 - val_loss: 1.0127 - val_accuracy: 0.6403
Epoch 115/1000
782/782 [==============================] - 4s 5ms/step - loss: 1.1322 - accuracy: 0.5973 - val_loss: 0.9782 - val_accuracy: 0.6494
Epoch 116/1000
782/782 [==============================] - 3s 4ms/step - loss: 1.1216 - accuracy: 0.6030 - val_loss: 1.0388 - val_accuracy: 0.6333
Epoch 117/1000
782/782 [==============================] - 4s 5ms/step - loss: 1.1257 - accuracy: 0.6024 - val_loss: 0.9928 - val_accuracy: 0.6459
Epoch 118/1000
782/782 [==============================] - 4s 5ms/step - loss: 1.1204 - accuracy: 0.6003 - val_loss: 0.9668 - val_accuracy: 0.6590
Epoch 119/1000
782/782 [==============================] - 4s 5ms/step - loss: 1.1220 - accuracy: 0.6027 - val_loss: 0.9947 - val_accuracy: 

782/782 [==============================] - 3s 4ms/step - loss: 0.9858 - accuracy: 0.6533 - val_loss: 0.8963 - val_accuracy: 0.6815
Epoch 226/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9770 - accuracy: 0.6576 - val_loss: 0.8980 - val_accuracy: 0.6839
Epoch 227/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9859 - accuracy: 0.6555 - val_loss: 0.8326 - val_accuracy: 0.7038
Epoch 228/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9734 - accuracy: 0.6602 - val_loss: 0.8563 - val_accuracy: 0.6915
Epoch 229/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.9757 - accuracy: 0.6591 - val_loss: 0.8390 - val_accuracy: 0.7020
Epoch 230/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.9766 - accuracy: 0.6565 - val_loss: 0.8630 - val_accuracy: 0.6919
Epoch 231/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.9777 - accuracy: 0.6558 - val_loss: 0.8310 - val_accuracy: 

782/782 [==============================] - 3s 4ms/step - loss: 0.8989 - accuracy: 0.6902 - val_loss: 0.8212 - val_accuracy: 0.7071
Epoch 338/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9084 - accuracy: 0.6850 - val_loss: 0.8267 - val_accuracy: 0.7095
Epoch 339/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.9044 - accuracy: 0.6859 - val_loss: 0.8031 - val_accuracy: 0.7150
Epoch 340/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.9017 - accuracy: 0.6860 - val_loss: 0.8214 - val_accuracy: 0.7097
Epoch 341/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.9020 - accuracy: 0.6876 - val_loss: 0.7740 - val_accuracy: 0.7277
Epoch 342/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.9081 - accuracy: 0.6848 - val_loss: 0.7659 - val_accuracy: 0.7295
Epoch 343/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.9057 - accuracy: 0.6878 - val_loss: 0.7835 - val_accuracy: 

782/782 [==============================] - 3s 4ms/step - loss: 0.8634 - accuracy: 0.7011 - val_loss: 0.7659 - val_accuracy: 0.7294
Epoch 450/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8645 - accuracy: 0.7015 - val_loss: 0.7783 - val_accuracy: 0.7276
Epoch 451/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.8654 - accuracy: 0.7018 - val_loss: 0.7396 - val_accuracy: 0.7422
Epoch 452/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8630 - accuracy: 0.7037 - val_loss: 0.8184 - val_accuracy: 0.7142
Epoch 453/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.8657 - accuracy: 0.6987 - val_loss: 0.7348 - val_accuracy: 0.7435
Epoch 454/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.8669 - accuracy: 0.7014 - val_loss: 0.7777 - val_accuracy: 0.7276
Epoch 455/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.8678 - accuracy: 0.7019 - val_loss: 0.7363 - val_accuracy: 

782/782 [==============================] - 4s 5ms/step - loss: 0.8415 - accuracy: 0.7091 - val_loss: 0.7170 - val_accuracy: 0.7512
Epoch 562/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8459 - accuracy: 0.7079 - val_loss: 0.7413 - val_accuracy: 0.7432
Epoch 563/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.8390 - accuracy: 0.7105 - val_loss: 0.7346 - val_accuracy: 0.7460
Epoch 564/1000
782/782 [==============================] - 4s 4ms/step - loss: 0.8473 - accuracy: 0.7066 - val_loss: 0.7065 - val_accuracy: 0.7569
Epoch 565/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8379 - accuracy: 0.7116 - val_loss: 0.7530 - val_accuracy: 0.7393
Epoch 566/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.8475 - accuracy: 0.7067 - val_loss: 0.7167 - val_accuracy: 0.7497
Epoch 567/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.8412 - accuracy: 0.7103 - val_loss: 0.7336 - val_accuracy: 

782/782 [==============================] - 4s 5ms/step - loss: 0.8321 - accuracy: 0.7129 - val_loss: 0.7158 - val_accuracy: 0.7521
Epoch 674/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8261 - accuracy: 0.7165 - val_loss: 0.7324 - val_accuracy: 0.7420
Epoch 675/1000
782/782 [==============================] - 4s 4ms/step - loss: 0.8299 - accuracy: 0.7145 - val_loss: 0.7018 - val_accuracy: 0.7559
Epoch 676/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8266 - accuracy: 0.7147 - val_loss: 0.7141 - val_accuracy: 0.7522
Epoch 677/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8270 - accuracy: 0.7124 - val_loss: 0.7169 - val_accuracy: 0.7502
Epoch 678/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8261 - accuracy: 0.7166 - val_loss: 0.7395 - val_accuracy: 0.7424
Epoch 679/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.8284 - accuracy: 0.7150 - val_loss: 0.6861 - val_accuracy: 

782/782 [==============================] - 3s 4ms/step - loss: 0.8097 - accuracy: 0.7215 - val_loss: 0.6942 - val_accuracy: 0.7582
Epoch 786/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.8148 - accuracy: 0.7191 - val_loss: 0.7148 - val_accuracy: 0.7523
Epoch 787/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.8199 - accuracy: 0.7174 - val_loss: 0.7395 - val_accuracy: 0.7417
Epoch 788/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.8215 - accuracy: 0.7157 - val_loss: 0.7304 - val_accuracy: 0.7461
Epoch 789/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8187 - accuracy: 0.7178 - val_loss: 0.7098 - val_accuracy: 0.7562
Epoch 790/1000
782/782 [==============================] - 3s 4ms/step - loss: 0.8166 - accuracy: 0.7193 - val_loss: 0.7063 - val_accuracy: 0.7586
Epoch 791/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8254 - accuracy: 0.7176 - val_loss: 0.7233 - val_accuracy: 

782/782 [==============================] - 4s 5ms/step - loss: 0.8094 - accuracy: 0.7234 - val_loss: 0.6923 - val_accuracy: 0.7636
Epoch 898/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8080 - accuracy: 0.7223 - val_loss: 0.6862 - val_accuracy: 0.7652
Epoch 899/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8071 - accuracy: 0.7236 - val_loss: 0.6936 - val_accuracy: 0.7605
Epoch 900/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8052 - accuracy: 0.7219 - val_loss: 0.6997 - val_accuracy: 0.7568
Epoch 901/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8107 - accuracy: 0.7224 - val_loss: 0.7060 - val_accuracy: 0.7568
Epoch 902/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8104 - accuracy: 0.7208 - val_loss: 0.7016 - val_accuracy: 0.7552
Epoch 903/1000
782/782 [==============================] - 4s 4ms/step - loss: 0.8099 - accuracy: 0.7208 - val_loss: 0.6825 - val_accuracy: 

In [109]:
# load the pre-saved model
model_10 = load_model('model_10_tiny_Base-CubeAI_6Conv.h5')

# test the accuracy of the model on the test set
loss, accuracy = model_10.evaluate(x_test, y_test)

# print the accuracy of the model on the test set
print('Test accuracy:', accuracy)

313/313 [==============================] - 1s 2ms/step - loss: 0.6847 - accuracy: 0.7631
Test accuracy: 0.7631000280380249


In [110]:
# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model_10)

# Convert the model to the TensorFlow Lite format with quantization
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

tflite_model_name = 'model_10_tiny_Base-CubeAI_6Conv_PTQ'
        
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce full-int8 quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name + '.tflite')
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of model_10_tiny_Base-CubeAI_6Conv_PTQ is {}%".format(accuracy_score*100))

INFO:tensorflow:Assets written to: /tmp/tmplofc0ni7/assets


INFO:tensorflow:Assets written to: /tmp/tmplofc0ni7/assets
/home/themandalorian/anaconda3/envs/ml-on-mc/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-23 13:57:54.772483: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-23 13:57:54.772510: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-23 13:57:54.773330: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmplofc0ni7
2023-05-23 13:57:54.778386: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-23 13:57:54.778411: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmplofc0ni7
2023-05-23 13:57:54.793306: I tensorflow/

== Input details ==
name: serving_default_conv2d_537_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.int8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.int8'>
Accuracy of model_10_tiny_Base-CubeAI_6Conv_PTQ is 76.29%


In [111]:
model_10 = Sequential()
model_10.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

# Convert the model to a quantization aware model
quant_aware_model = tfmot.quantization.keras.quantize_model(model_10)

quant_aware_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint_callback = ModelCheckpoint(filepath='model_10_tiny_Base-CubeAI_6Conv_QAT.h5', save_freq='epoch')

quant_aware_model.summary()

# Train and evaluate the quantization aware model
quant_aware_model.fit(x_train,y_train, batch_size=64,epochs=1000,
                      callbacks=[checkpoint_callback],validation_data=(x_test, y_test))

Model: "sequential_90"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLay  (None, 32, 32, 3)        3         
 er)                                                             
                                                                 
 quant_conv2d_543 (QuantizeW  (None, 32, 32, 8)        243       
 rapperV2)                                                       
                                                                 
 quant_conv2d_544 (QuantizeW  (None, 32, 32, 8)        603       
 rapperV2)                                                       
                                                                 
 quant_max_pooling2d_278 (Qu  (None, 16, 16, 8)        1         
 antizeWrapperV2)                                                
                                                                 
 quant_dropout_363 (Quantize  (None, 16, 16, 8)      

782/782 [==============================] - 7s 9ms/step - loss: 1.1497 - accuracy: 0.5919 - val_loss: 1.0622 - val_accuracy: 0.6257
Epoch 88/1000
782/782 [==============================] - 7s 9ms/step - loss: 1.1441 - accuracy: 0.5948 - val_loss: 1.1046 - val_accuracy: 0.6153
Epoch 89/1000
782/782 [==============================] - 7s 9ms/step - loss: 1.1462 - accuracy: 0.5938 - val_loss: 1.0272 - val_accuracy: 0.6365
Epoch 90/1000
782/782 [==============================] - 7s 9ms/step - loss: 1.1411 - accuracy: 0.5973 - val_loss: 1.0313 - val_accuracy: 0.6321
Epoch 91/1000
782/782 [==============================] - 7s 10ms/step - loss: 1.1398 - accuracy: 0.5965 - val_loss: 1.0011 - val_accuracy: 0.6479
Epoch 92/1000
782/782 [==============================] - 8s 10ms/step - loss: 1.1420 - accuracy: 0.5972 - val_loss: 1.0872 - val_accuracy: 0.6174
Epoch 93/1000
782/782 [==============================] - 8s 10ms/step - loss: 1.1342 - accuracy: 0.5982 - val_loss: 1.0547 - val_accuracy: 0.6

782/782 [==============================] - 7s 9ms/step - loss: 1.0502 - accuracy: 0.6313 - val_loss: 1.0097 - val_accuracy: 0.6545
Epoch 144/1000
782/782 [==============================] - 7s 9ms/step - loss: 1.0430 - accuracy: 0.6316 - val_loss: 0.9696 - val_accuracy: 0.6623
Epoch 145/1000
782/782 [==============================] - 7s 9ms/step - loss: 1.0438 - accuracy: 0.6342 - val_loss: 0.9071 - val_accuracy: 0.6815
Epoch 146/1000
782/782 [==============================] - 7s 9ms/step - loss: 1.0417 - accuracy: 0.6336 - val_loss: 0.9396 - val_accuracy: 0.6683
Epoch 147/1000
782/782 [==============================] - 7s 9ms/step - loss: 1.0431 - accuracy: 0.6318 - val_loss: 0.9921 - val_accuracy: 0.6560
Epoch 148/1000
782/782 [==============================] - 7s 9ms/step - loss: 1.0398 - accuracy: 0.6347 - val_loss: 0.9785 - val_accuracy: 0.6589
Epoch 149/1000
782/782 [==============================] - 7s 9ms/step - loss: 1.0373 - accuracy: 0.6361 - val_loss: 0.8880 - val_accuracy: 

782/782 [==============================] - 7s 10ms/step - loss: 0.9512 - accuracy: 0.6682 - val_loss: 0.8514 - val_accuracy: 0.7026
Epoch 256/1000
782/782 [==============================] - 7s 9ms/step - loss: 0.9577 - accuracy: 0.6657 - val_loss: 0.8363 - val_accuracy: 0.7090
Epoch 257/1000
782/782 [==============================] - 7s 9ms/step - loss: 0.9515 - accuracy: 0.6683 - val_loss: 0.8163 - val_accuracy: 0.7138
Epoch 258/1000
782/782 [==============================] - 7s 9ms/step - loss: 0.9506 - accuracy: 0.6695 - val_loss: 0.8224 - val_accuracy: 0.7127
Epoch 259/1000
782/782 [==============================] - 8s 10ms/step - loss: 0.9521 - accuracy: 0.6689 - val_loss: 0.9151 - val_accuracy: 0.6841
Epoch 260/1000
782/782 [==============================] - 8s 10ms/step - loss: 0.9524 - accuracy: 0.6681 - val_loss: 0.8613 - val_accuracy: 0.7025
Epoch 261/1000
782/782 [==============================] - 8s 10ms/step - loss: 0.9513 - accuracy: 0.6702 - val_loss: 0.8097 - val_accura

782/782 [==============================] - 8s 10ms/step - loss: 0.9277 - accuracy: 0.6762 - val_loss: 0.7912 - val_accuracy: 0.7262
Epoch 312/1000
782/782 [==============================] - 7s 10ms/step - loss: 0.9260 - accuracy: 0.6771 - val_loss: 0.7980 - val_accuracy: 0.7208
Epoch 313/1000
782/782 [==============================] - 7s 9ms/step - loss: 0.9264 - accuracy: 0.6780 - val_loss: 0.8091 - val_accuracy: 0.7188
Epoch 314/1000
782/782 [==============================] - 7s 9ms/step - loss: 0.9282 - accuracy: 0.6778 - val_loss: 0.8067 - val_accuracy: 0.7211
Epoch 315/1000
782/782 [==============================] - 7s 9ms/step - loss: 0.9279 - accuracy: 0.6747 - val_loss: 0.8812 - val_accuracy: 0.7036
Epoch 316/1000
782/782 [==============================] - 7s 9ms/step - loss: 0.9247 - accuracy: 0.6784 - val_loss: 0.8003 - val_accuracy: 0.7212
Epoch 317/1000
782/782 [==============================] - 8s 10ms/step - loss: 0.9271 - accuracy: 0.6773 - val_loss: 0.8249 - val_accurac

782/782 [==============================] - 7s 9ms/step - loss: 0.8908 - accuracy: 0.6924 - val_loss: 0.7912 - val_accuracy: 0.7284
Epoch 424/1000
782/782 [==============================] - 7s 9ms/step - loss: 0.8935 - accuracy: 0.6908 - val_loss: 0.7725 - val_accuracy: 0.7303
Epoch 425/1000
782/782 [==============================] - 7s 9ms/step - loss: 0.8941 - accuracy: 0.6909 - val_loss: 0.7746 - val_accuracy: 0.7291
Epoch 426/1000
782/782 [==============================] - 7s 9ms/step - loss: 0.8869 - accuracy: 0.6903 - val_loss: 0.7704 - val_accuracy: 0.7319
Epoch 427/1000
782/782 [==============================] - 7s 9ms/step - loss: 0.8877 - accuracy: 0.6931 - val_loss: 0.7563 - val_accuracy: 0.7392
Epoch 428/1000
782/782 [==============================] - 7s 9ms/step - loss: 0.8868 - accuracy: 0.6934 - val_loss: 0.8113 - val_accuracy: 0.7162
Epoch 429/1000
782/782 [==============================] - 8s 10ms/step - loss: 0.8957 - accuracy: 0.6888 - val_loss: 0.7907 - val_accuracy:

Epoch 535/1000
782/782 [==============================] - 12s 16ms/step - loss: 0.8671 - accuracy: 0.7010 - val_loss: 0.8189 - val_accuracy: 0.7231
Epoch 536/1000
782/782 [==============================] - 13s 16ms/step - loss: 0.8698 - accuracy: 0.7004 - val_loss: 0.7513 - val_accuracy: 0.7357
Epoch 537/1000
782/782 [==============================] - 13s 16ms/step - loss: 0.8680 - accuracy: 0.7015 - val_loss: 0.7499 - val_accuracy: 0.7462
Epoch 538/1000
782/782 [==============================] - 13s 16ms/step - loss: 0.8698 - accuracy: 0.6998 - val_loss: 0.7437 - val_accuracy: 0.7445
Epoch 539/1000
782/782 [==============================] - 12s 16ms/step - loss: 0.8702 - accuracy: 0.6996 - val_loss: 0.8073 - val_accuracy: 0.7202
Epoch 540/1000
782/782 [==============================] - 12s 16ms/step - loss: 0.8727 - accuracy: 0.6989 - val_loss: 0.7429 - val_accuracy: 0.7454
Epoch 541/1000
782/782 [==============================] - 12s 16ms/step - loss: 0.8657 - accuracy: 0.7016 - val_

782/782 [==============================] - 12s 16ms/step - loss: 0.8483 - accuracy: 0.7089 - val_loss: 0.7500 - val_accuracy: 0.7413
Epoch 646/1000
782/782 [==============================] - 12s 16ms/step - loss: 0.8542 - accuracy: 0.7067 - val_loss: 0.7376 - val_accuracy: 0.7442
Epoch 647/1000
782/782 [==============================] - 13s 16ms/step - loss: 0.8486 - accuracy: 0.7097 - val_loss: 0.7751 - val_accuracy: 0.7321
Epoch 648/1000
782/782 [==============================] - 13s 16ms/step - loss: 0.8551 - accuracy: 0.7049 - val_loss: 0.7722 - val_accuracy: 0.7355
Epoch 649/1000
782/782 [==============================] - 13s 16ms/step - loss: 0.8514 - accuracy: 0.7047 - val_loss: 0.7326 - val_accuracy: 0.7457
Epoch 650/1000
782/782 [==============================] - 12s 15ms/step - loss: 0.8511 - accuracy: 0.7085 - val_loss: 0.7473 - val_accuracy: 0.7444
Epoch 651/1000
782/782 [==============================] - 13s 16ms/step - loss: 0.8537 - accuracy: 0.7049 - val_loss: 0.7542 - 

In [5]:
# evaluate the model on the test set
quant_loss, quant_acc = quant_aware_model.evaluate(x_test, y_test, verbose=0)
print('Quantization aware training loss: ', quant_loss)
print('Quantization aware training accuracy: ', quant_acc)

# convert the QAT model to a fully quantized model using TFLite
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant_int8_qat = converter.convert()

open("model_10_tiny_Base-CubeAI_6Conv_QAT.tflite", "wb").write(tflite_model_quant_int8_qat)

tflite_interpreter = tf.lite.Interpreter("model_10_tiny_Base-CubeAI_6Conv_QAT.tflite")
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of model_10_tiny_Base-CubeAI_6Conv_QAT is {}%".format(accuracy_score*100))

== Input details ==
name: serving_default_conv2d_543_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.uint8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.uint8'>
Accuracy of model_10_tiny_Base-CubeAI_6Conv_QAT is 75.28%


In [114]:
c_model_name = 'cifar10_tiny_base_cubeAI_6conv_ptq'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model, c_model_name))

In [115]:
c_model_name = 'cifar10_tiny_base_cubeAI_6conv_qat'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model_quant_int8_qat, c_model_name))

In [14]:
model_10 = Sequential()
model_10.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

# compile model
opt = SGD(lr=0.001, momentum=0.9)

model_10.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint_callback = ModelCheckpoint(filepath='model_10_tiny_Base-CubeAI_8Conv.h5', save_freq='epoch')

model_10.summary()

# Train and evaluate the quantization aware model
model_10.fit(x_train,y_train, batch_size=64,epochs=1000,callbacks=[checkpoint_callback],
             validation_data=(x_test, y_test))

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_50 (Conv2D)          (None, 32, 32, 8)         224       
                                                                 
 conv2d_51 (Conv2D)          (None, 32, 32, 8)         584       
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 16, 16, 8)        0         
 g2D)                                                            
                                                                 
 dropout_40 (Dropout)        (None, 16, 16, 8)         0         
                                                                 
 conv2d_52 (Conv2D)          (None, 16, 16, 16)        1168      
                                                                 
 conv2d_53 (Conv2D)          (None, 16, 16, 16)        2320      
                                                      

2023-05-23 16:57:15.621367: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


  1/782 [..............................] - ETA: 25:38 - loss: 2.5346 - accuracy: 0.1719

2023-05-23 16:57:16.584354: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


782/782 [==============================] - 6s 6ms/step - loss: 2.2665 - accuracy: 0.1374 - val_loss: 2.2481 - val_accuracy: 0.1701
Epoch 2/1000
782/782 [==============================] - 4s 5ms/step - loss: 2.1528 - accuracy: 0.1751 - val_loss: 2.1623 - val_accuracy: 0.1708
Epoch 3/1000
782/782 [==============================] - 4s 5ms/step - loss: 2.0497 - accuracy: 0.1885 - val_loss: 2.1042 - val_accuracy: 0.2002
Epoch 4/1000
782/782 [==============================] - 4s 5ms/step - loss: 1.9846 - accuracy: 0.2098 - val_loss: 1.9553 - val_accuracy: 0.2765
Epoch 5/1000
782/782 [==============================] - 4s 5ms/step - loss: 1.9345 - accuracy: 0.2393 - val_loss: 2.0230 - val_accuracy: 0.2392
Epoch 6/1000
782/782 [==============================] - 4s 5ms/step - loss: 1.8985 - accuracy: 0.2564 - val_loss: 1.9828 - val_accuracy: 0.2578
Epoch 7/1000
782/782 [==============================] - 4s 5ms/step - loss: 1.8720 - accuracy: 0.2679 - val_loss: 1.8547 - val_accuracy: 0.2988
Epoch

782/782 [==============================] - 4s 5ms/step - loss: 1.1426 - accuracy: 0.6002 - val_loss: 0.9943 - val_accuracy: 0.6379
Epoch 114/1000
782/782 [==============================] - 4s 5ms/step - loss: 1.1369 - accuracy: 0.6037 - val_loss: 0.9645 - val_accuracy: 0.6487
Epoch 115/1000
782/782 [==============================] - 4s 5ms/step - loss: 1.1381 - accuracy: 0.6048 - val_loss: 0.9696 - val_accuracy: 0.6497
Epoch 116/1000
782/782 [==============================] - 4s 5ms/step - loss: 1.1403 - accuracy: 0.6012 - val_loss: 0.9760 - val_accuracy: 0.6474
Epoch 117/1000
782/782 [==============================] - 4s 5ms/step - loss: 1.1335 - accuracy: 0.6057 - val_loss: 0.9689 - val_accuracy: 0.6495
Epoch 118/1000
782/782 [==============================] - 4s 5ms/step - loss: 1.1349 - accuracy: 0.6057 - val_loss: 0.9516 - val_accuracy: 0.6591
Epoch 119/1000
782/782 [==============================] - 4s 5ms/step - loss: 1.1342 - accuracy: 0.6050 - val_loss: 0.9740 - val_accuracy: 

782/782 [==============================] - 4s 5ms/step - loss: 0.9867 - accuracy: 0.6613 - val_loss: 0.8370 - val_accuracy: 0.7044
Epoch 226/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.9909 - accuracy: 0.6600 - val_loss: 0.8388 - val_accuracy: 0.7008
Epoch 227/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.9879 - accuracy: 0.6610 - val_loss: 0.8492 - val_accuracy: 0.6955
Epoch 228/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.9881 - accuracy: 0.6614 - val_loss: 0.8815 - val_accuracy: 0.6826
Epoch 229/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.9882 - accuracy: 0.6612 - val_loss: 0.8346 - val_accuracy: 0.7027
Epoch 230/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.9818 - accuracy: 0.6629 - val_loss: 0.8345 - val_accuracy: 0.7027
Epoch 231/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.9785 - accuracy: 0.6647 - val_loss: 0.8556 - val_accuracy: 

782/782 [==============================] - 4s 5ms/step - loss: 0.8971 - accuracy: 0.6979 - val_loss: 0.7546 - val_accuracy: 0.7354
Epoch 338/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8895 - accuracy: 0.6983 - val_loss: 0.7541 - val_accuracy: 0.7340
Epoch 339/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8967 - accuracy: 0.6966 - val_loss: 0.7428 - val_accuracy: 0.7391
Epoch 340/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.9002 - accuracy: 0.6939 - val_loss: 0.8001 - val_accuracy: 0.7226
Epoch 341/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8988 - accuracy: 0.6967 - val_loss: 0.7627 - val_accuracy: 0.7299
Epoch 342/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8924 - accuracy: 0.6988 - val_loss: 0.7817 - val_accuracy: 0.7271
Epoch 343/1000
782/782 [==============================] - 4s 5ms/step - loss: 0.8902 - accuracy: 0.6972 - val_loss: 0.7838 - val_accuracy: 

In [4]:
# load the pre-saved model
model_10 = load_model('model_10_tiny_Base-CubeAI_8Conv.h5')

# test the accuracy of the model on the test set
loss, accuracy = model_10.evaluate(x_test, y_test)

# print the accuracy of the model on the test set
print('Test accuracy:', accuracy)

2023-05-23 22:18:37.744146: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-23 22:18:37.744469: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-23 22:18:37.744683: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-23 22:18:37.745141: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-23 22:18:37.745367: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

 65/313 [=====>........................] - ETA: 0s - loss: 0.6527 - accuracy: 0.7865

2023-05-23 22:18:39.712779: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


313/313 [==============================] - 2s 2ms/step - loss: 0.6420 - accuracy: 0.7794
Test accuracy: 0.7793999910354614


In [5]:
# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model_10)

# Convert the model to the TensorFlow Lite format with quantization
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

tflite_model_name = 'model_10_tiny_Base-CubeAI_8Conv_PTQ'
        
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce full-int8 quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name + '.tflite')
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of model_10_tiny_Base-CubeAI_8Conv_PTQ is {}%".format(accuracy_score*100))

INFO:tensorflow:Assets written to: /tmp/tmple61pd3b/assets


INFO:tensorflow:Assets written to: /tmp/tmple61pd3b/assets
/home/themandalorian/anaconda3/envs/ml-on-mc/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-23 22:18:57.946320: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-23 22:18:57.946348: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-23 22:18:57.946911: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmple61pd3b
2023-05-23 22:18:57.949795: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-23 22:18:57.949813: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmple61pd3b
2023-05-23 22:18:57.958419: I tensorflow/

== Input details ==
name: serving_default_conv2d_50_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.int8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.int8'>
Accuracy of model_10_tiny_Base-CubeAI_8Conv_PTQ is 78.03%


In [6]:
c_model_name = 'cifar10_tiny_base_cubeAI_8conv_ptq'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model, c_model_name))

In [4]:
model_10 = Sequential()
model_10.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

# compile model
opt = SGD(lr=0.001, momentum=0.9)

# Convert the model to a quantization aware model
quant_aware_model = tfmot.quantization.keras.quantize_model(model_10)

quant_aware_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint_callback = ModelCheckpoint(filepath='model_10_tiny_Base-CubeAI_8Conv_QAT.h5', save_freq='epoch')

quant_aware_model.summary()

# Train and evaluate the quantization aware model
quant_aware_model.fit(x_train,y_train, batch_size=64,epochs=400,
                      callbacks=[checkpoint_callback],validation_data=(x_test, y_test))

2023-05-29 18:26:28.498525: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-29 18:26:28.498850: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-29 18:26:28.499121: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-29 18:26:28.499635: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-29 18:26:28.499886: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLay  (None, 32, 32, 3)        3         
 er)                                                             
                                                                 
 quant_conv2d (QuantizeWrapp  (None, 32, 32, 8)        243       
 erV2)                                                           
                                                                 
 quant_conv2d_1 (QuantizeWra  (None, 32, 32, 8)        603       
 pperV2)                                                         
                                                                 
 quant_max_pooling2d (Quanti  (None, 16, 16, 8)        1         
 zeWrapperV2)                                                    
                                                                 
 quant_dropout (QuantizeWrap  (None, 16, 16, 8)        1

2023-05-29 18:26:31.960138: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-05-29 18:26:32.998263: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


782/782 [==============================] - 13s 13ms/step - loss: 2.3150 - accuracy: 0.1058 - val_loss: 2.2965 - val_accuracy: 0.1384
Epoch 2/400
782/782 [==============================] - 10s 13ms/step - loss: 2.2584 - accuracy: 0.1490 - val_loss: 2.2000 - val_accuracy: 0.1791
Epoch 3/400
782/782 [==============================] - 10s 12ms/step - loss: 2.1548 - accuracy: 0.1706 - val_loss: 2.1087 - val_accuracy: 0.2011
Epoch 4/400
782/782 [==============================] - 10s 12ms/step - loss: 2.0739 - accuracy: 0.1844 - val_loss: 2.0102 - val_accuracy: 0.2213
Epoch 5/400
782/782 [==============================] - 10s 13ms/step - loss: 2.0231 - accuracy: 0.1949 - val_loss: 2.0698 - val_accuracy: 0.1865
Epoch 6/400
782/782 [==============================] - 10s 13ms/step - loss: 1.9960 - accuracy: 0.1972 - val_loss: 1.9706 - val_accuracy: 0.2247
Epoch 7/400
782/782 [==============================] - 10s 13ms/step - loss: 1.9694 - accuracy: 0.2103 - val_loss: 1.9499 - val_accuracy: 0.23

782/782 [==============================] - 10s 13ms/step - loss: 1.3550 - accuracy: 0.5185 - val_loss: 1.2437 - val_accuracy: 0.5590
Epoch 58/400
782/782 [==============================] - 10s 13ms/step - loss: 1.3456 - accuracy: 0.5227 - val_loss: 1.2281 - val_accuracy: 0.5582
Epoch 59/400
782/782 [==============================] - 10s 13ms/step - loss: 1.3426 - accuracy: 0.5224 - val_loss: 1.1889 - val_accuracy: 0.5719
Epoch 60/400
782/782 [==============================] - 10s 13ms/step - loss: 1.3373 - accuracy: 0.5230 - val_loss: 1.2047 - val_accuracy: 0.5657
Epoch 61/400
782/782 [==============================] - 10s 13ms/step - loss: 1.3306 - accuracy: 0.5300 - val_loss: 1.2002 - val_accuracy: 0.5743
Epoch 62/400
782/782 [==============================] - 10s 13ms/step - loss: 1.3269 - accuracy: 0.5307 - val_loss: 1.2032 - val_accuracy: 0.5692
Epoch 63/400
782/782 [==============================] - 10s 13ms/step - loss: 1.3174 - accuracy: 0.5320 - val_loss: 1.2294 - val_accuracy

...

Epoch 225/400
782/782 [==============================] - 20s 26ms/step - loss: 1.0098 - accuracy: 0.6525 - val_loss: 0.8542 - val_accuracy: 0.7012
Epoch 226/400
782/782 [==============================] - 21s 27ms/step - loss: 1.0132 - accuracy: 0.6482 - val_loss: 0.8784 - val_accuracy: 0.6884
Epoch 227/400
782/782 [==============================] - 20s 25ms/step - loss: 1.0087 - accuracy: 0.6507 - val_loss: 0.8882 - val_accuracy: 0.6848
Epoch 228/400
782/782 [==============================] - 20s 26ms/step - loss: 1.0120 - accuracy: 0.6507 - val_loss: 0.8876 - val_accuracy: 0.6882
Epoch 229/400
782/782 [==============================] - 20s 25ms/step - loss: 1.0166 - accuracy: 0.6491 - val_loss: 0.8774 - val_accuracy: 0.6884
Epoch 230/400
782/782 [==============================] - 21s 27ms/step - loss: 1.0135 - accuracy: 0.6497 - val_loss: 0.8814 - val_accuracy: 0.6926
Epoch 231/400
782/782 [==============================] - 20s 26ms/step - loss: 1.0070 - accuracy: 0.6511 - val_loss: 0

Epoch 281/400
782/782 [==============================] - 20s 25ms/step - loss: 0.9716 - accuracy: 0.6672 - val_loss: 0.8034 - val_accuracy: 0.7211
Epoch 282/400
782/782 [==============================] - 21s 26ms/step - loss: 0.9635 - accuracy: 0.6673 - val_loss: 0.8009 - val_accuracy: 0.7238
Epoch 283/400
782/782 [==============================] - 20s 25ms/step - loss: 0.9748 - accuracy: 0.6652 - val_loss: 0.8035 - val_accuracy: 0.7168
Epoch 284/400
782/782 [==============================] - 21s 27ms/step - loss: 0.9673 - accuracy: 0.6685 - val_loss: 0.8441 - val_accuracy: 0.7053
Epoch 285/400
782/782 [==============================] - 20s 26ms/step - loss: 0.9665 - accuracy: 0.6686 - val_loss: 0.8082 - val_accuracy: 0.7165
Epoch 286/400
782/782 [==============================] - 21s 26ms/step - loss: 0.9648 - accuracy: 0.6676 - val_loss: 0.8248 - val_accuracy: 0.7121
Epoch 287/400
782/782 [==============================] - 20s 25ms/step - loss: 0.9662 - accuracy: 0.6701 - val_loss: 0

Epoch 337/400
782/782 [==============================] - 22s 28ms/step - loss: 0.9326 - accuracy: 0.6802 - val_loss: 0.7761 - val_accuracy: 0.7245
Epoch 338/400
782/782 [==============================] - 20s 26ms/step - loss: 0.9390 - accuracy: 0.6762 - val_loss: 0.7747 - val_accuracy: 0.7302
Epoch 339/400
782/782 [==============================] - 20s 25ms/step - loss: 0.9264 - accuracy: 0.6803 - val_loss: 0.7821 - val_accuracy: 0.7228
Epoch 340/400
782/782 [==============================] - 20s 25ms/step - loss: 0.9349 - accuracy: 0.6778 - val_loss: 0.7829 - val_accuracy: 0.7253
Epoch 341/400
782/782 [==============================] - 21s 26ms/step - loss: 0.9406 - accuracy: 0.6784 - val_loss: 0.7787 - val_accuracy: 0.7247
Epoch 342/400
782/782 [==============================] - 21s 27ms/step - loss: 0.9270 - accuracy: 0.6798 - val_loss: 0.7740 - val_accuracy: 0.7232
Epoch 343/400
782/782 [==============================] - 20s 25ms/step - loss: 0.9305 - accuracy: 0.6817 - val_loss: 0

Epoch 393/400
782/782 [==============================] - 20s 26ms/step - loss: 0.9042 - accuracy: 0.6925 - val_loss: 0.7464 - val_accuracy: 0.7365
Epoch 394/400
782/782 [==============================] - 20s 26ms/step - loss: 0.9008 - accuracy: 0.6916 - val_loss: 0.7796 - val_accuracy: 0.7265
Epoch 395/400
782/782 [==============================] - 20s 25ms/step - loss: 0.9121 - accuracy: 0.6873 - val_loss: 0.7469 - val_accuracy: 0.7423
Epoch 396/400
782/782 [==============================] - 20s 25ms/step - loss: 0.9116 - accuracy: 0.6884 - val_loss: 0.7961 - val_accuracy: 0.7205
Epoch 397/400
782/782 [==============================] - 20s 26ms/step - loss: 0.9043 - accuracy: 0.6891 - val_loss: 0.7655 - val_accuracy: 0.7350
Epoch 398/400
782/782 [==============================] - 21s 26ms/step - loss: 0.9035 - accuracy: 0.6906 - val_loss: 0.7818 - val_accuracy: 0.7282
Epoch 399/400
782/782 [==============================] - 21s 27ms/step - loss: 0.9082 - accuracy: 0.6903 - val_loss: 0

KeyboardInterrupt: 

In [6]:
# evaluate the model on the test set
quant_loss, quant_acc = quant_aware_model.evaluate(x_test, y_test, verbose=0)
print('Quantization aware training loss: ', quant_loss)
print('Quantization aware training accuracy: ', quant_acc)

# convert the QAT model to a fully quantized model using TFLite
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant_int8_qat = converter.convert()

open("model_10_tiny_Base-CubeAI_8Conv_QAT.tflite", "wb").write(tflite_model_quant_int8_qat)

Quantization aware training loss:  0.7397522330284119
Quantization aware training accuracy:  0.7407000064849854


INFO:tensorflow:Assets written to: /tmp/tmpv7ii9uuo/assets


INFO:tensorflow:Assets written to: /tmp/tmpv7ii9uuo/assets
/home/themandalorian/anaconda3/envs/ml-on-mc/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-29 20:13:56.401946: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-29 20:13:56.402077: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-29 20:13:56.402478: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpv7ii9uuo
2023-05-29 20:13:56.460846: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-29 20:13:56.460897: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpv7ii9uuo
2023-05-29 20:13:56.608866: I tensorflow/

106976

In [7]:
tflite_interpreter = tf.lite.Interpreter("model_10_tiny_Base-CubeAI_8Conv_QAT.tflite")
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of model_10_tiny_Base-CubeAI_8Conv_QAT is {}%".format(accuracy_score*100))

c_model_name = 'cifar10_tiny_base_cubeAI_8conv_qat'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model_quant_int8_qat, c_model_name))

== Input details ==
name: serving_default_conv2d_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.uint8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.uint8'>
Accuracy of model_10_tiny_Base-CubeAI_8Conv_QAT is 74.15%


In [14]:
model_10 = Sequential()
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

# compile model
opt = SGD(lr=0.001, momentum=0.9)

model_10.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint_callback = ModelCheckpoint(filepath='model_10_tiny_Base-CubeAI_Triple_10Conv.h5', save_freq='epoch')

model_10.summary()

# Train and evaluate the quantization aware model
model_10.fit(x_train,y_train, batch_size=64,epochs=400,callbacks=[checkpoint_callback],
             validation_data=(x_test, y_test))

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_56 (Conv2D)          (None, 32, 32, 16)        448       
                                                                 
 conv2d_57 (Conv2D)          (None, 32, 32, 16)        2320      
                                                                 
 conv2d_58 (Conv2D)          (None, 32, 32, 16)        2320      
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 16, 16, 16)       0         
 g2D)                                                            
                                                                 
 dropout_34 (Dropout)        (None, 16, 16, 16)        0         
                                                                 
 conv2d_59 (Conv2D)          (None, 16, 16, 32)        4640      
                                                      

782/782 [==============================] - 11s 14ms/step - loss: 1.0686 - accuracy: 0.6205 - val_loss: 0.9727 - val_accuracy: 0.6550
Epoch 33/400
782/782 [==============================] - 11s 15ms/step - loss: 1.0624 - accuracy: 0.6236 - val_loss: 0.9762 - val_accuracy: 0.6534
Epoch 34/400
782/782 [==============================] - 10s 13ms/step - loss: 1.0521 - accuracy: 0.6285 - val_loss: 0.9435 - val_accuracy: 0.6645
Epoch 35/400
782/782 [==============================] - 11s 15ms/step - loss: 1.0350 - accuracy: 0.6364 - val_loss: 0.9722 - val_accuracy: 0.6566
Epoch 36/400
782/782 [==============================] - 10s 13ms/step - loss: 1.0276 - accuracy: 0.6373 - val_loss: 0.9679 - val_accuracy: 0.6602
Epoch 37/400
782/782 [==============================] - 11s 15ms/step - loss: 1.0194 - accuracy: 0.6418 - val_loss: 0.9281 - val_accuracy: 0.6699
Epoch 38/400
782/782 [==============================] - 11s 14ms/step - loss: 1.0096 - accuracy: 0.6414 - val_loss: 0.9870 - val_accuracy

782/782 [==============================] - 10s 13ms/step - loss: 0.7200 - accuracy: 0.7500 - val_loss: 0.7142 - val_accuracy: 0.7499
Epoch 89/400
782/782 [==============================] - 11s 15ms/step - loss: 0.7154 - accuracy: 0.7533 - val_loss: 0.7346 - val_accuracy: 0.7430
Epoch 90/400
782/782 [==============================] - 11s 15ms/step - loss: 0.7161 - accuracy: 0.7512 - val_loss: 0.6927 - val_accuracy: 0.7618
Epoch 91/400
782/782 [==============================] - 10s 13ms/step - loss: 0.7117 - accuracy: 0.7538 - val_loss: 0.7215 - val_accuracy: 0.7505
Epoch 92/400
782/782 [==============================] - 11s 15ms/step - loss: 0.7082 - accuracy: 0.7543 - val_loss: 0.7093 - val_accuracy: 0.7527
Epoch 93/400
782/782 [==============================] - 11s 14ms/step - loss: 0.7015 - accuracy: 0.7569 - val_loss: 0.7105 - val_accuracy: 0.7549
Epoch 94/400
782/782 [==============================] - 11s 15ms/step - loss: 0.6958 - accuracy: 0.7616 - val_loss: 0.6837 - val_accuracy

In [29]:
# load the pre-saved model
model_10 = load_model('model_10_tiny_Base-CubeAI_Triple_10Conv.h5')

# test the accuracy of the model on the test set
loss, accuracy = model_10.evaluate(x_test, y_test)

# print the accuracy of the model on the test set
print('Test accuracy:', accuracy)

In [30]:
# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model_10)

# Convert the model to the TensorFlow Lite format with quantization
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

tflite_model_name = 'model_10_tiny_Base-CubeAI_Triple_10Conv_PTQ'
        
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce full-int8 quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name + '.tflite')
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of model_10_tiny_Base-CubeAI_Triple_10Conv_PTQ is {}%".format(accuracy_score*100))

In [31]:
c_model_name = 'cifar10_tiny_base_cubeAI_triple_10conv_ptq'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model, c_model_name))

In [32]:
model_10 = Sequential()
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

# compile model
opt = SGD(lr=0.001, momentum=0.9)

# Convert the model to a quantization aware model
quant_aware_model = tfmot.quantization.keras.quantize_model(model_10)

quant_aware_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint_callback = ModelCheckpoint(filepath='model_10_tiny_Base-CubeAI_Triple_10Conv_QAT.h5', save_freq='epoch')

quant_aware_model.summary()

# Train and evaluate the quantization aware model
quant_aware_model.fit(x_train,y_train, batch_size=64,epochs=400,
                      callbacks=[checkpoint_callback],validation_data=(x_test, y_test))

In [33]:
# evaluate the model on the test set
quant_loss, quant_acc = quant_aware_model.evaluate(x_test, y_test, verbose=0)
print('Quantization aware training loss: ', quant_loss)
print('Quantization aware training accuracy: ', quant_acc)

# convert the QAT model to a fully quantized model using TFLite
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant_int8_qat = converter.convert()

open("model_10_tiny_Base-CubeAI_Triple_10Conv_QAT.tflite", "wb").write(tflite_model_quant_int8_qat)

In [34]:
tflite_interpreter = tf.lite.Interpreter("model_10_tiny_Base-CubeAI_Triple_10Conv_QAT.tflite")
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of model_10_tiny_Base-CubeAI_Triple_10Conv_QAT is {}%".format(accuracy_score*100))

c_model_name = 'cifar10_tiny_base_cubeAI_triple_10conv_qat'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model_quant_int8_qat, c_model_name))